In [34]:
!pip install anytree

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.3 MB/s eta 0:00:00


In [36]:
import openai
import os
import json
from anytree import Node, RenderTree

openai.api_key  = os.environ.get('API_KEY')

In [ ]:
def get_completion(prompt, model="gpt-4"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

### Prompt for previous step (feature 1: topics)

In [87]:
transcript = f"""
Participant 1: "The role of social media in political campaigns is a subject that has gained significant attention in recent years. It has become a powerful tool for politicians to engage with voters and spread their message. However, there are concerns about the spread of misinformation and the manipulation of public opinion through targeted ads." Participant 2: "I agree that social media provides a platform for political candidates to connect with a wider audience and mobilize support. However, the lack of regulation and transparency in political advertising on these platforms is a major issue that needs to be addressed." Participant 3: "I believe that social media has democratized political discourse and allowed marginalized voices to be heard. It provides a platform for grassroots movements and enables citizens to participate in political discussions like never before. We should focus on educating users about media literacy and critical thinking to combat misinformation." Participant 4: "While social media has its benefits, the algorithms used by these platforms tend to create echo chambers and reinforce existing biases. We need better regulation to ensure that diverse viewpoints are represented and to prevent the manipulation of public opinion through targeted content."
Participant 1: "What are we gonna do about the rising population? We can't keep up."  Participant 2: "We could start by improving our public services, specifically health and education." Participant 3: "I think immigration policies should be stricter. There are too many people coming in."  Participant 4: "Wait, but what if we focus on creating more jobs? More people means more workforce." Participant 5: "Let's not forget about housing. We need more and affordable homes." Participant 6: "The environment! We can't ignore the impact of population growth on nature." Participant 7: "What about promoting family planning programs? It has worked in some countries."
"""


In [66]:
#Old fake prompt
prompt = f"""
You are an assistant for group discussions, specialized on keeping track and documenting the discussion,/
that is, the topics discussed, the viewpoints/positions on each topic and the arguments/explanations given in support of each viewpoint./
Identify the topics discussed in the discussion transcript, delimited with triple backticks.
Proceed in the following steps:

Step 1: Are there one or several topics being discussed the transcript? A discussion topic refers to "the subject being talked about".
Step 2: If there is only one discussion topic, summarize the topic in 1 keyword, or more if necessary. Create a dictionary with the identified and summarized topic as key and the transcript as value.
        Format the dictionary in JSON-format.
        If there are several discussion topics, summarize each topic in 1 keyword, or more if necessary. Create a dictionary with all the identified and summarized topics as keys and the corresponding excerpts from the transcript as value.
        Format the dictionary in JSON-format.
Only include the dictionary in your response.
```{transcript}```
"""


In [88]:
prompt = f"""
You are an assistant for group discussions, specializing in keeping track of and documenting the discussion,/
that is, the topics discussed, the viewpoints/positions on each topic, and the arguments/explanations given in support of each viewpoint./
Identify the topics discussed in the discussion transcript, delimited with triple backticks.
Check the following steps but don’t print each step only print what is asked:
Step 1: Identify the topics discussed in the discussion transcript. Take the time to read the complete transcript, don’t
take words as a whole argument, analyze the complete argument and then decide on the topic.\
Step 2: Analyze each topic (if there is more than one), and merge those into only one main topic.
If there are topics that are as important as the main topic, display them as a second topic but only if it is linked with the
main topic of discussion in the transcript. Only in this case, you can display more topics.
Step 3: Extract the statements related to each topic.
Step 4: State the necessary main topics of the entire transcript in a concise, descriptive sentence,
in up to 3 words for each topic.\
Step 5: Provide the output in a JSON format where the key is the topic and the value is a list of statements made by the
participants.
Review transcript: '''{transcript}'''
"""

In [89]:
topics = get_completion(prompt)

In [90]:
topics_json = json.loads(topics)
print(topics_json)
print(topics_json.keys())

{'Social Media in Politics': ['The role of social media in political campaigns is a subject that has gained significant attention in recent years. It has become a powerful tool for politicians to engage with voters and spread their message. However, there are concerns about the spread of misinformation and the manipulation of public opinion through targeted ads.', 'I agree that social media provides a platform for political candidates to connect with a wider audience and mobilize support. However, the lack of regulation and transparency in political advertising on these platforms is a major issue that needs to be addressed.', 'I believe that social media has democratized political discourse and allowed marginalized voices to be heard. It provides a platform for grassroots movements and enables citizens to participate in political discussions like never before. We should focus on educating users about media literacy and critical thinking to combat misinformation.', 'While social media h

### Prompt for feature 2: viewpoints

In [91]:
topics_json.keys()

dict_keys(['Social Media in Politics', 'Population Growth'])

In [82]:
def get_viewpoints_by_topic(topics,transcript):
    result = {}
    for topic, excerpt in topics.items():
        prompt = f"""
        You are an assistant for group discussions, specialized on keeping track and documenting the discussion,
        that is, the topics discussed, the viewpoints/positions on each topic and the arguments/explanations given in support of each viewpoint.
        For the topic "{topic}"elimited by triple hashtags, analyse the corresponding excerpt from the discussion. 
        Your task is to identify all the viewpoints expressed on the topic. 

        Proceed according to the following steps:

        Step 1: In the below discussion transcript, delimited by triple backticks, 
                locate the excerpt corresponding to the topic and consider the rest 
                of the transcript as context for the subsequent steps.
        Step 2: Are there one or several viewpoints being expressed in the excerpt? 
                A "viewpoint" refers to "one's perspective of opinion on a particular topic".
        Step 3: If there is only one viewpoint, summarize the viewpoint in 3 keywords max, 
                more keywords only if necessary to fully grasp the viewpoint. Viewpoint keywords 
                should be expressed as noun phrases that describe the viewpoint in a depersonalized manner. 
                For example, instead of “Supports Renewables”, the viewpoint keyword should be “Support for Renewables”. 
                Instead of “Believes in Traditional Energy”, the viewpoint keyword should be “Belief in Traditional Energy”.
                
                If there are several viewpoints, summarize each viewpoint in 3 keywords max, more keywords only if 
                necessary to fully grasp the topic. Viewpoint keywords should be expressed as noun phrases that describe 
                the viewpoint in a depersonalized manner, as explained in the instruction for one viewpoint.                 
        Step 4: Disregard viewpoints that are not relevant to the current topic or more relevant to another topic. Only if a viewpoint is equally relevant to multiple topics, include it under all relevant topics. 
        Step 5: Identify any linkages between viewpoints that build upon each other or propose solutions to identified issues. For instance, if a viewpoint such as 'Media literacy and critical thinking' is expressed as a solution to the issue identified in another viewpoint like 'Concerns about misinformation', classify it as a sub-viewpoint of the latter. Represent these sub-viewpoints appropriately within the hierarchical structure of the result dictionary.
        Step 6: Identify viewpoints that convey essentially the same stance on the topic. For example, viewpoints like 'Lack of regulation and transparency' and 'Need for better regulation' express similar concerns regarding the need for increased regulation in the domain. In such cases, merge these viewpoints into a single unified viewpoint that encapsulates both perspectives. Ensure this is reflected in the summary of viewpoints in the result dictionary.
        Step 7: Append the topics-dictionary with the identified and summarized viewpoints as sub-keys and the corresponding discussion excerpts as values.Format the dictionary in JSON-format. 
        
        Only include the appended dictionary in your response. 
        
        ```{transcript}```
        """  
        viewpoints = get_completion(prompt)
        viewpoints_json = json.loads(viewpoints)
        result.update(viewpoints_json)
    return result

In [92]:
viewpoints_by_topics = get_viewpoints_by_topic(topics_json,transcript)

In [93]:
viewpoints_by_topics

{'Social Media in Politics': {'Powerful tool for engagement': 'The role of social media in political campaigns is a subject that has gained significant attention in recent years. It has become a powerful tool for politicians to engage with voters and spread their message.',
  'Concerns about misinformation': {'Lack of regulation and transparency': 'I agree that social media provides a platform for political candidates to connect with a wider audience and mobilize support. However, the lack of regulation and transparency in political advertising on these platforms is a major issue that needs to be addressed.',
   'Media literacy and critical thinking': 'I believe that social media has democratized political discourse and allowed marginalized voices to be heard. It provides a platform for grassroots movements and enables citizens to participate in political discussions like never before. We should focus on educating users about media literacy and critical thinking to combat misinformatio

In [61]:
def json_to_tree(viewpoints_by_topics):
    def process_viewpoint(name, viewpoint, parent=None):
        node = Node(name, parent=parent)
        if isinstance(viewpoint, dict):
            for sub_viewpoint, content in viewpoint.items():
                process_viewpoint(sub_viewpoint, content, parent=node)
        return node

    for topic, viewpoints in viewpoints_by_topics.items():
        topic_node = Node(topic)
        for viewpoint, content in viewpoints.items():
            process_viewpoint(viewpoint, content, parent=topic_node)
        for pre, _, node in RenderTree(topic_node):
            print("%s%s" % (pre, node.name))




In [94]:
json_to_tree(viewpoints_by_topics)

Social Media in Politics
├── Powerful tool for engagement
├── Concerns about misinformation
│   ├── Lack of regulation and transparency
│   └── Media literacy and critical thinking
└── Echo chambers and biases
Population Growth
├── Improving Public Services
├── Stricter Immigration Policies
├── Job Creation
├── Housing
├── Environmental Impact
└── Family Planning Programs


In [64]:
def json_to_mermaid(viewpoints_by_topics):
    def process_viewpoint(name, viewpoint, parent=None):
        nonlocal counter
        node_id = f"N{counter}"
        counter += 1
        nodes[node_id] = name
        if parent is not None:
            edges.append(f"{parent} --> {node_id}")
        if isinstance(viewpoint, dict):
            for sub_viewpoint, content in viewpoint.items():
                process_viewpoint(sub_viewpoint, content, parent=node_id)

    counter = 0
    nodes = {}
    edges = []
    for topic, viewpoints in viewpoints_by_topics.items():
        topic_id = f"N{counter}"
        nodes[topic_id] = topic
        counter += 1
        for viewpoint, content in viewpoints.items():
            process_viewpoint(viewpoint, content, parent=topic_id)

    mermaid_graph = "graph TB\n"
    for node_id, node_name in nodes.items():
        mermaid_graph += f'  {node_id}[{node_name}]\n'
    for edge in edges:
        mermaid_graph += f'  {edge}\n'
    for i in range(len(edges)):
        mermaid_graph += f"  linkStyle {i} stroke:#2ecd71,stroke-width:2px;\n"
    return mermaid_graph


In [95]:
print(json_to_mermaid(viewpoints_by_topics))

graph TB
  N0[Social Media in Politics]
  N1[Powerful tool for engagement]
  N2[Concerns about misinformation]
  N3[Lack of regulation and transparency]
  N4[Media literacy and critical thinking]
  N5[Echo chambers and biases]
  N6[Population Growth]
  N7[Improving Public Services]
  N8[Stricter Immigration Policies]
  N9[Job Creation]
  N10[Housing]
  N11[Environmental Impact]
  N12[Family Planning Programs]
  N0 --> N1
  N0 --> N2
  N2 --> N3
  N2 --> N4
  N0 --> N5
  N6 --> N7
  N6 --> N8
  N6 --> N9
  N6 --> N10
  N6 --> N11
  N6 --> N12
  linkStyle 0 stroke:#2ecd71,stroke-width:2px;
  linkStyle 1 stroke:#2ecd71,stroke-width:2px;
  linkStyle 2 stroke:#2ecd71,stroke-width:2px;
  linkStyle 3 stroke:#2ecd71,stroke-width:2px;
  linkStyle 4 stroke:#2ecd71,stroke-width:2px;
  linkStyle 5 stroke:#2ecd71,stroke-width:2px;
  linkStyle 6 stroke:#2ecd71,stroke-width:2px;
  linkStyle 7 stroke:#2ecd71,stroke-width:2px;
  linkStyle 8 stroke:#2ecd71,stroke-width:2px;
  linkStyle 9 stroke:#2ecd71,